# JavaUDF TinyBar Chart

Our goal in this exercise is to create a UDF, permanently registered, that displays visual spark bars (miniature within-cell bar charts) available for reporting on any interface.  We will do this by:

- [ ] Creating a class that renders Unicode little bar charts (▁▆██)
- [ ] Registering this UDF permanently with Snowflake (TINY_BARS)
- [ ] Test a DataFrame using TINY_BARS

![](../assets/tinybars_overview.gif)

## Connect to Snowflake

In [ ]:
import com.snowflake.snowpark._
import com.snowflake.snowpark.functions._
import com.snowflake.snowpark.types._

In [ ]:
// Set connection properties built in de_snowpark/A-Dataframes/01-Sessions.ipynb
val pwd = sys.env.get("PWD").fold("")(_.toString)
val filename = s"$pwd/de_snowpark/connect.properties"

val session = Session.builder.configFile(s"$filename").create

In [ ]:
// Change to the MODELED schema in your animal [LOGIN]_DB 
session.sql("use schema modeled").collect

In [ ]:
import session.implicits._

In [ ]:
// Create a Snowflake internal stage that will be used by our Java UDFs
session.sql("create stage if not exists RAW.JAVA_UDF_STAGE").collect

## Create the Class that Renders Tiny Bar Chart

Create our class that draws, using unicode characters : ▁▆██▅█▄ bar charts

In [ ]:
class tinyChart() extends Serializable {
  def tinyBars = (vals: Array[String]) => {
    val dVals = vals.map(_.toDouble)
    val max = dVals.max
    // creating little bar charts like this "▁▂▃▄▅▆▇█" + max
    var bars = new StringBuilder
    dVals.foreach( (v: Double) => {
        val percent = v / max.toFloat

        if (percent < 0.125)                          { bars ++= "▁"}
        else if (percent >= 0.125 && percent < 0.250) { bars ++= "▂"}
        else if (percent >= 0.250 && percent < 0.375) { bars ++= "▃"}
        else if (percent >= 0.375 && percent < 0.500) { bars ++= "▄"}
        else if (percent >= 0.500 && percent < 0.625) { bars ++= "▅"}
        else if (percent >= 0.625 && percent < 0.750) { bars ++= "▆"}
        else if (percent >= 0.750 && percent < 0.875) { bars ++= "▇"}
        else                                          { bars ++= "█"}
    })
    bars.toString
  }
}

In [ ]:
// Test locally... 
println(new tinyChart().tinyBars(Array("3","5", "2")))
println(new tinyChart().tinyBars(Array("1","12", "15", "17")))
println(new tinyChart().tinyBars(Array("1","2","3","4","5","6","7","8","7","6","5","4","3","2","1")))

### Progress: Check

- [X] Creating a class that renders Unicode little bar charts (▁▆██)
- [ ] Registering this UDF permanently with Snowflake (TINY_BARS)
- [ ] Test a DataFrame using TINY_BARS

## Register UDF in Snowflake

Let's create the `MODELED.TINY_BARS` UDF in Snowflake.  We'll use `registerPermanent` to collect, upload our .jar and register a permanent named JavaUDF to Snowflake.

In [ ]:
session.udf.registerPermanent("modeled.TINY_BARS", new tinyChart().tinyBars, "RAW.JAVA_UDF_STAGE")

### Progress: Check

- [X] Creating a class that renders Unicode little bar charts (▁▆██)
- [X] Registering this UDF permanently with Snowflake (TINY_BARS)
- [ ] Test a DataFrame using TINY_BARS

Now, let's try out this new UDF. We do so by creating a new DataFrame from an array, and then invoking our function via callUDF to return a new DataFrame with an additional column - the output being a little bar chart representation of the data.

In [ ]:
session
    .sql("select array_construct('1','5','17','10','20') as input_values")
    .withColumn("barchart", callUDF("modeled.TINY_BARS", col("input_values")))
.show

### Progress: Check

- [X] Creating a class that renders Unicode little bar charts (▁▆██)
- [X] Registering this UDF permanently with Snowflake (TINY_BARS)
- [X] Test a DataFrame using TINY_BARS